# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [44]:
#!pip install selenium
from selenium import webdriver # Criar navegador
from selenium.webdriver.common.by import By # Localizar elementos (os itens de um site)
from selenium.webdriver.common.keys import Keys # Permite clicar teclas no teclado

navegador = webdriver.Chrome() # Abre navegador

#.send_keys()
#.click()
#.get_attribute()

# Passo 1: Entrar no Google
navegador.get("https://www.google.com/")

# Passo 2: Pesquisar a cotação do dolar
navegador.find_element(By.XPATH,
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação do dolar")

navegador.find_element(By.XPATH,
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

# Passo 3: Pegar cotação do dolar
cotacao_dolar = navegador.find_element(By.XPATH,
                                       '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
print(cotacao_dolar)

# Passo 4: Pegar cotação do euro
navegador.get("https://www.google.com/")

navegador.find_element(By.XPATH,
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação do euro")

navegador.find_element(By.XPATH,
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_euro = navegador.find_element(By.XPATH,
                                       '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
print(cotacao_euro)

# Passo 5: Pegar cotação do ouro
navegador.get("https://www.melhorcambio.com/ouro-hoje")

cotacao_ouro = navegador.find_element(By.XPATH,'//*[@id="comercial"]').get_attribute("value")
cotacao_ouro = cotacao_ouro.replace(",",".")
print(cotacao_ouro)

5.138
5.4072
306.28


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [45]:
# Passo 6: Atualizar minha base de dados com as novas cotações
import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [46]:
# Atualizar a cotação de acordo com a moeda correspondente
# Dolar -> tabela.loc[linha, coluna]
tabela.loc[tabela["Moeda"]=="Dólar","Cotação"] = float(cotacao_dolar) # float tranforma texto em número com casa decimal
# Euro
tabela.loc[tabela["Moeda"]=="Euro","Cotação"] = float(cotacao_euro) # float tranforma texto em número com casa decimal
# Ouro
tabela.loc[tabela["Moeda"]=="Ouro","Cotação"] = float(cotacao_ouro) # float tranforma texto em número com casa decimal

# Atualizar preço de compra = preço original * cotação
tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]

# Atualizar preço de venda = preço de compra * margem
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]

tabela["Preço de Venda atualizada"] = tabela["Preço de Compra"] * tabela["Margem"] # Se não encontar a coluna, então Python cria uma nova

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda,Preço de Venda atualizada
0,Câmera Canon,999.99,Dólar,5.1380,5137.94862,1.40,7193.128068,7193.128068
1,Carro Renault,4500.00,Euro,5.4072,24332.40000,2.00,48664.800000,48664.800000
2,Notebook Dell,899.99,Dólar,5.1380,4624.14862,1.70,7861.052654,7861.052654
3,IPhone,799.00,Dólar,5.1380,4105.26200,1.70,6978.945400,6978.945400
4,Carro Fiat,3000.00,Euro,5.4072,16221.60000,1.90,30821.040000,30821.040000
5,Celular Xiaomi,480.48,Dólar,5.1380,2468.70624,2.00,4937.412480,4937.412480
6,Joia 20g,20.00,Ouro,306.2800,6125.60000,1.15,7044.440000,7044.440000


### Agora vamos exportar a nova base de preços atualizada

In [47]:
# Exporatar a base de vendas
tabela.to_excel("Produtos.xlsx",index=False)
navegador.quit() # Fecha o navegador 